In [1]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "Patchy consolidation on the left lower lobe"
)

matches = matcher(doc)

print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 0


In [2]:
# Write a pattern for adjective plus one or two nouns
consolidation_pattern = [{"POS": "ADJ"}, {"LEMMA": "consolidation"}]
lobe_pattern = [{"POS": "ADJ"}, {"LEMMA": "lobe"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("ADJ_CONSOLIDATION", None, consolidation_pattern)
matcher.add("LOBE", None, lobe_pattern)
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 2
Match found: Patchy consolidation
Match found: lower lobe


In [3]:
# Now let's use OP to capture multiple ADJECTIVES
lobe_pattern = [{"POS": "ADJ", "OP" : "+"}, {"LEMMA": "lobe"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("LOBE", None, lobe_pattern)
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 3
Match found: Patchy consolidation
Match found: lower lobe
Match found: left lower lobe


In [4]:
doc2 = nlp(
    "Infiltrates seen suggesting possibility of pneumonia"
)

In [5]:
# Now let's use OP to capture multiple ADJECTIVES
infiltrate_pattern = [{"LEMMA": "infiltrate"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("INFILTRATE", None, infiltrate_pattern)
matches = matcher(doc2)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc2[start:end].text)

Total matches found: 1
Match found: Infiltrates
